In [1]:
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
import os
import sys
import logging
from openpyxl import load_workbook
from datetime import datetime, date
import pandas as pd
import threading
from pathos.multiprocessing import ProcessingPool as Pool
from concurrent.futures import ProcessPoolExecutor, as_completed
import concurrent.futures
from trade.assets.Stock import Stock
from trade.helpers.helper import generate_option_tick_new
from trade.assets.rates import get_risk_free_rate_helper
from trade.helpers.helper import IV_handler, time_distance_helper, binomial_implied_vol, wait_for_response
from trade.helpers.helper import extract_numeric_value, change_to_last_busday
from trade.helpers.Logging import setup_logger
from trade.assets.Calculate import Calculate
from trade.helpers.Context import Context
from dbase.DataAPI.ThetaData import retrieve_ohlc, retrieve_quote_rt, retrieve_eod_ohlc, resample, retrieve_quote
from dbase.DataAPI.Organizers import generate_optionData_to_save, Calc_Risks
from dbase.database.SQLHelpers import store_SQL_data_Insert_Ignore, query_database, dynamic_batch_update
from trade.helpers.decorators import log_error, log_error_with_stack
from trade.helpers.types import OptionModelAttributes
from abc import ABC, abstractmethod


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-07 22:54:01 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [6]:
__all__ = [
    'DataManagerBase',
    'OptionDataManager',
    'SpotDataManager',
    'VolDataManager',
    'GreeksDataManager',
    'AttributionDataManager',
]

In [7]:

## Format of tables is: database_name.table_name
TABLES = {
    'eod':{
        'attribution': 'securities_master.attribution_eod',
        'spot': 'securities_master.temp_options_eod',
        'vol': 'securities_master.temp_options_eod',
        'greeks': 'securities_master.temp_options_eod',
    },
    'intra':{
        'attribution': 'securities_master.attribution_intra',
        'spot': 'securities_master.temp_options_intra',
        'vol': 'securities_master.temp_options_intra',
        'greeks': 'securities_master.temp_options_intra',
    }
}


In [22]:
class OptionDataManager:
    def __init__(self, 
                 symbol: str,
                 exp: str | datetime,
                 right: str,
                 strike: float,
                 default_fill: str = 'midpoint') -> None:
        """
        Returns an object for querying data

        Params:
        symbol: Underlier symbol
        exp: expiration
        right: Put(P) or Call (C)
        strike: Option Strike
        default_fill: How to fill zero values for close. 'midpoint' or 'weighted_midpoint'
        """
        assert isinstance(strike, float), f"Strike has to be type float, recieved {type(strike)}"
        if default_fill not in ['midpoint', 'weighted_midpoint', None]:
            raise ValueError("Expected default_fill to be one of: 'midpoint', 'weighted_midpoint', None ")
        self.exp = exp
        self.symbol = symbol
        self.right = right.upper()
        self.strike = strike
        self.default_fill = default_fill
        self.opttick = generate_option_tick_new(symbol, right, exp, strike)
        self.Stock = Stock(symbol, run_chain = False)
    

    def get_timeseries(self, 
                       start: str | datetime, 
                       end: str | datetime,
                       interval: str = '1d',
                       type_: str = 'spot',
                       model: str = 'bs') -> pd.DataFrame:
        
        
        ## Organize inputs
        start = pd.to_datetime(start)
        end = pd.to_datetime(end)
        ivl_str, ivl_int = extract_numeric_value(interval)
        greek_names = ['vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greek', 'greeks']


        ## Assert inputs
        if type_ not in ['spot', 'vol', 'vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greeks', 'greek', 'attribution']:
            raise ValueError("Expected type_ to be one of: ['spot', 'vol', 'vega', 'vanna', 'volga', 'delta', 'gamma', 'theta', 'rho', 'greeks', 'greek', 'attribution']")
        if model not in ['bs', 'bt', 'mc', 'bsm']: ## Only Black Scholes, binomial tree, monte carlo
            raise ValueError("Expected model to be one of: ['bs', 'bt', 'mc', 'bsm']")
        
        if ivl_str.lower() not in ['d', 'w','q','y', 'h'] and ivl_str != 'M': ## Want to avoid minute data
            raise ValueError("Expected interval to be one of: ['d', 'w','q','y' 'M']")
        
        if ivl_str == 'm': ## Minute data not available
            raise AttributeError("Minute data currently unavailable, please go higher")
        

        ## Determine aggregation
        if ivl_str == 'h':
            agg = 'intra'
        else:
            agg = 'eod'
        
        ## Table to query, picking based on interval & type
        if type_ in greek_names:
            table = TABLES[agg]['greek']
        else:
            database, table = TABLES[agg][type_].split('.')
        print(database, table)
            



    def __verify_data_completeness(self, data: pd.DataFrame, interval_type: str) -> 'Something':
        """
        Verify that the data is complete
        """
        pass



class SpotDataManager:
    pass

class VolDataManager:
    pass

class GreeksDataManager:
    pass

class AttributionDataManager:
    pass



In [12]:
OptionDataManager?

Init signature:
OptionDataManager(
    symbol: str,
    exp: str | datetime.datetime,
    right: str,
    strike: float,
    default_fill: str = 'midpoint',
) -> None
Docstring:      <no docstring>
Init docstring:
Returns an object for querying data

Params:
symbol: Underlier symbol
exp: expiration
right: Put(P) or Call (C)
strike: Option Strike
default_fill: How to fill zero values for close. 'midpoint' or 'weighted_midpoint'
Type:           type
Subclasses:     

In [23]:
manager = OptionDataManager('AAPL', '2021-09-17', 'C', 145.0)

In [26]:
manager.get_timeseries('2021-09-01', '2021-09-10', interval='1h', type_='attribution', model='bs')

securities_master attribution_intra
